# TF-IDF Text Classification with N-grams and Naive Bayes

This notebook demonstrates:
1. **TF-IDF Vectorization with N-grams** using scikit-learn
2. **Naive Bayes Classification** for Tamil news articles
3. **Model Evaluation** with comprehensive metrics

**Features:**
- N-gram TF-IDF features (unigrams + bigrams, max 10,000 features)
- Document frequency filtering (min_df=3, max_df=0.8)
- Multinomial Naive Bayes classifier
- Dual task: Category classification and Sentiment classification
- Comparison with unigram-only approach

**Dataset:** Tamil news articles with categories and processed text

**N-gram Benefits:**
- Captures word sequences (e.g., "இலங்கை அரசு")
- Better context understanding
- More discriminative features

## 1. Import Required Libraries

In [ ]:
import pandas as pd
import numpy as np
import pickle
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)

print("Libraries imported successfully!")
print("N-gram tokenization enabled ✓")

## 2. Load the Cleaned Data

In [ ]:
# Load the processed data from preprocessing notebook
df = pd.read_csv('output/processed_data.csv')

print(f"Dataset shape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")
print(f"\nFirst few rows:")
df.head()

In [ ]:
# Check data distribution
print("Category distribution:")
print(df['category'].value_counts())
print(f"\nTotal samples: {len(df)}")
print(f"Missing values:\n{df.isnull().sum()}")

## 3. Prepare Text Data for TF-IDF

We'll use the `cleaned_title` column which contains preprocessed Tamil text.

In [ ]:
# Select the text column and target variable
documents = df['cleaned_title'].fillna('').tolist()
labels = df['category'].tolist()

print(f"Total documents: {len(documents)}")
print(f"Total labels: {len(labels)}")
print(f"\nSample document: {documents[0]}")
print(f"Sample label: {labels[0]}")

## 4. TF-IDF Vectorization with N-grams

**TF-IDF (Term Frequency-Inverse Document Frequency)** converts text into numerical features:
- **TF**: Measures how frequently a term appears in a document
- **IDF**: Measures how important a term is across all documents
- **TF-IDF**: Combines both to get weighted features

**N-gram Enhancement:**
- **Unigrams (1-gram)**: Single words ("இலங்கை", "அரசு")
- **Bigrams (2-gram)**: Word pairs ("இலங்கை அரசு")
- **ngram_range=(1,2)**: Combines both unigrams and bigrams

**Parameters:**
- `max_features=10000`: Keep only top 10,000 most important n-grams
- `min_df=3`: Ignore n-grams appearing in less than 3 documents
- `max_df=0.8`: Ignore n-grams appearing in more than 80% of documents
- `ngram_range=(1,2)`: Use unigrams and bigrams

In [ ]:
# Initialize TF-IDF vectorizer with n-grams
tfidf_vectorizer = TfidfVectorizer(
    max_features=10000,
    min_df=3,
    max_df=0.8,
    ngram_range=(1, 2),  # Unigrams + Bigrams
    token_pattern=r'\S+',  # Split on whitespace
    dtype=np.float32
)

# Fit and transform the documents
print("Creating TF-IDF matrix with n-grams...")
tfidf_matrix = tfidf_vectorizer.fit_transform(documents)

print(f"\n✓ TF-IDF Matrix created: {tfidf_matrix.shape}")
print(f"  Documents: {tfidf_matrix.shape[0]:,}")
print(f"  N-gram Features: {tfidf_matrix.shape[1]:,}")
print(f"  Matrix type: {type(tfidf_matrix)}")
print(f"  Data type: {tfidf_matrix.dtype}")
print(f"  Sparsity: {(1 - tfidf_matrix.nnz / (tfidf_matrix.shape[0] * tfidf_matrix.shape[1])) * 100:.2f}%")

## 5. N-gram Feature Examples

Let's look at some example features to understand what n-grams capture.

In [ ]:
# Get feature names (n-grams)
feature_names = tfidf_vectorizer.get_feature_names_out()

print(f"Total features: {len(feature_names):,}")
print(f"\nSample unigrams (single words):")
unigrams = [f for f in feature_names[:50] if ' ' not in f]
print(unigrams[:10])

print(f"\nSample bigrams (word pairs):")
bigrams = [f for f in feature_names if ' ' in f]
print(bigrams[:10])

print(f"\nFeature type breakdown:")
n_unigrams = sum(1 for f in feature_names if ' ' not in f)
n_bigrams = sum(1 for f in feature_names if ' ' in f)
print(f"  Unigrams: {n_unigrams:,} ({n_unigrams/len(feature_names)*100:.1f}%)")
print(f"  Bigrams: {n_bigrams:,} ({n_bigrams/len(feature_names)*100:.1f}%)")

## 6. Prepare Data for Machine Learning

Split the data into training (80%) and testing (20%) sets with stratified sampling.

In [ ]:
# Use TF-IDF matrix as features
X = tfidf_matrix
y = np.array(labels)

# Split data: 80% train, 20% test with stratification
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training set size: {X_train.shape[0]} samples")
print(f"Test set size: {X_test.shape[0]} samples")
print(f"N-gram feature dimension: {X_train.shape[1]}")
print(f"\nTraining set label distribution:")
print(pd.Series(y_train).value_counts())
print(f"\nTest set label distribution:")
print(pd.Series(y_test).value_counts())

## 7. Train Naive Bayes Model

**Naive Bayes** is a probabilistic classifier based on Bayes' theorem:
- **Fast training and prediction**
- **Works well with text data**
- **Assumes independence between features**
- **MultinomialNB** is designed for discrete features like word counts/TF-IDF
- **Works effectively with n-gram features**

In [ ]:
# Initialize and train Naive Bayes model
print("Training Naive Bayes model with n-gram features...")
nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)

# Make predictions
nb_train_pred = nb_model.predict(X_train)
nb_test_pred = nb_model.predict(X_test)

print("✓ Naive Bayes training completed")
print(f"  Classes: {nb_model.classes_}")
print(f"  Number of classes: {len(nb_model.classes_)}")

## 8. Model Evaluation

Evaluate the model using multiple metrics:
- **Accuracy**: Overall correctness
- **Precision**: How many selected items are relevant
- **Recall**: How many relevant items are selected
- **F1-Score**: Harmonic mean of precision and recall

In [ ]:
def evaluate_model(y_true, y_pred, model_name, dataset_name):
    """
    Evaluate model performance with multiple metrics.
    """
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)
    
    print(f"\n{'='*60}")
    print(f"{model_name} - {dataset_name} Set")
    print(f"{'='*60}")
    print(f"Accuracy:  {accuracy:.4f} ({accuracy*100:.2f}%)")
    print(f"Precision: {precision:.4f} ({precision*100:.2f}%)")
    print(f"Recall:    {recall:.4f} ({recall*100:.2f}%)")
    print(f"F1-Score:  {f1:.4f} ({f1*100:.2f}%)")
    
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

# Evaluate on training and test sets
nb_train_metrics = evaluate_model(y_train, nb_train_pred, "Naive Bayes (N-gram)", "Training")
nb_test_metrics = evaluate_model(y_test, nb_test_pred, "Naive Bayes (N-gram)", "Test")

In [ ]:
# Detailed classification report
print("\nDetailed Classification Report (Test Set):")
print(classification_report(y_test, nb_test_pred, zero_division=0))

## 9. Confusion Matrix

Visualize the confusion matrix to see which categories are confused with each other.

In [ ]:
# Compute confusion matrix
nb_cm = confusion_matrix(y_test, nb_test_pred)
classes = sorted(list(set(y_test)))

# Plot confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(nb_cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=classes, yticklabels=classes, cbar_kws={'label': 'Count'})
plt.title('Confusion Matrix - Naive Bayes with N-grams (Category)', fontsize=14, fontweight='bold')
plt.xlabel('Predicted Label', fontsize=12)
plt.ylabel('True Label', fontsize=12)
plt.tight_layout()
plt.show()

## 10. Save Model and Vectorizer

In [ ]:
# Create directories if they don't exist
os.makedirs('models', exist_ok=True)
os.makedirs('reports', exist_ok=True)
os.makedirs('output', exist_ok=True)

# Save the trained model
with open('models/category_naive_bayes_tfidf_ngram.pkl', 'wb') as f:
    pickle.dump(nb_model, f)

# Save the TF-IDF vectorizer
with open('models/category_tfidf_vectorizer_ngram.pkl', 'wb') as f:
    pickle.dump(tfidf_vectorizer, f)

# Save evaluation report
import json

report = {
    'model': 'MultinomialNB',
    'vectorizer': 'TfidfVectorizer',
    'ngram_range': '(1,2)',
    'max_features': 10000,
    'train_metrics': nb_train_metrics,
    'test_metrics': nb_test_metrics,
    'classification_report': classification_report(y_test, nb_test_pred, output_dict=True, zero_division=0)
}

with open('reports/category_naive_bayes_tfidf_ngram_report.json', 'w') as f:
    json.dump(report, f, indent=2)

print("✓ Model saved to models/category_naive_bayes_tfidf_ngram.pkl")
print("✓ Vectorizer saved to models/category_tfidf_vectorizer_ngram.pkl")
print("✓ Evaluation report saved to reports/category_naive_bayes_tfidf_ngram_report.json")

---

# SENTIMENT CLASSIFICATION WITH N-GRAMS

---

## 11. Load Sentiment Dataset

In [ ]:
# Load sentiment data
df_sentiment = pd.read_csv('output/processed_sentiment_data.csv')

print(f"Sentiment Dataset shape: {df_sentiment.shape}")
print(f"\nSentiment distribution:")
print(df_sentiment['sentiment'].value_counts())

sentiment_documents = df_sentiment['tokenized_title'].fillna('').tolist()
sentiment_labels = df_sentiment['sentiment'].tolist()

print(f"\nTotal sentiment documents: {len(sentiment_documents)}")
print(f"Sample: {sentiment_documents[0]}")

## 12. TF-IDF Vectorization for Sentiment with N-grams

In [ ]:
# Initialize TF-IDF vectorizer for sentiment with n-grams
sentiment_tfidf_vectorizer = TfidfVectorizer(
    max_features=10000,
    min_df=3,
    max_df=0.8,
    ngram_range=(1, 2),  # Unigrams + Bigrams
    token_pattern=r'\S+',
    dtype=np.float32
)

# Fit and transform sentiment documents
print("Creating TF-IDF matrix for sentiment with n-grams...")
sentiment_tfidf_matrix = sentiment_tfidf_vectorizer.fit_transform(sentiment_documents)

print(f"\n✓ Sentiment TF-IDF Matrix: {sentiment_tfidf_matrix.shape}")
print(f"  Documents: {sentiment_tfidf_matrix.shape[0]:,}")
print(f"  N-gram Features: {sentiment_tfidf_matrix.shape[1]:,}")
print(f"  Sparsity: {(1 - sentiment_tfidf_matrix.nnz / (sentiment_tfidf_matrix.shape[0] * sentiment_tfidf_matrix.shape[1])) * 100:.2f}%")

## 13. Prepare Sentiment Data for Training

In [ ]:
X_sent = sentiment_tfidf_matrix
y_sent = np.array(sentiment_labels)

# Split data with stratification
X_sent_train, X_sent_test, y_sent_train, y_sent_test = train_test_split(
    X_sent, y_sent, test_size=0.2, random_state=42, stratify=y_sent
)

print(f"Sentiment Training set: {X_sent_train.shape[0]} samples")
print(f"Sentiment Test set: {X_sent_test.shape[0]} samples")
print(f"N-gram features: {X_sent_train.shape[1]}")
print(f"\nSentiment distribution (train):")
print(pd.Series(y_sent_train).value_counts())
print(f"\nSentiment distribution (test):")
print(pd.Series(y_sent_test).value_counts())

## 14. Train Sentiment Naive Bayes Model

In [ ]:
# Initialize and train Naive Bayes for sentiment
print("Training Sentiment Naive Bayes model with n-grams...")
sent_nb_model = MultinomialNB()
sent_nb_model.fit(X_sent_train, y_sent_train)

# Make predictions
sent_nb_train_pred = sent_nb_model.predict(X_sent_train)
sent_nb_test_pred = sent_nb_model.predict(X_sent_test)

print("✓ Sentiment Naive Bayes training completed")
print(f"  Classes: {sent_nb_model.classes_}")

## 15. Evaluate Sentiment Model

In [ ]:
# Evaluate sentiment model
sent_nb_train_metrics = evaluate_model(y_sent_train, sent_nb_train_pred, "Sentiment Naive Bayes (N-gram)", "Training")
sent_nb_test_metrics = evaluate_model(y_sent_test, sent_nb_test_pred, "Sentiment Naive Bayes (N-gram)", "Test")

In [ ]:
# Detailed classification report
print("\nDetailed Classification Report (Sentiment - Test Set):")
print(classification_report(y_sent_test, sent_nb_test_pred, zero_division=0))

## 16. Sentiment Confusion Matrix

In [ ]:
# Compute and plot sentiment confusion matrix
sent_nb_cm = confusion_matrix(y_sent_test, sent_nb_test_pred)
sent_classes = sorted(list(set(y_sent_test)))

plt.figure(figsize=(8, 6))
sns.heatmap(sent_nb_cm, annot=True, fmt='d', cmap='Greens', 
            xticklabels=sent_classes, yticklabels=sent_classes, cbar_kws={'label': 'Count'})
plt.title('Confusion Matrix - Naive Bayes with N-grams (Sentiment)', fontsize=14, fontweight='bold')
plt.xlabel('Predicted Label', fontsize=12)
plt.ylabel('True Label', fontsize=12)
plt.tight_layout()
plt.show()

## 17. Save Sentiment Model and Vectorizer

In [ ]:
# Save sentiment model
with open('models/sentiment_naive_bayes_tfidf_ngram.pkl', 'wb') as f:
    pickle.dump(sent_nb_model, f)

# Save sentiment vectorizer
with open('models/sentiment_tfidf_vectorizer_ngram.pkl', 'wb') as f:
    pickle.dump(sentiment_tfidf_vectorizer, f)

# Save evaluation report
sent_report = {
    'model': 'MultinomialNB',
    'vectorizer': 'TfidfVectorizer',
    'ngram_range': '(1,2)',
    'max_features': 10000,
    'train_metrics': sent_nb_train_metrics,
    'test_metrics': sent_nb_test_metrics,
    'classification_report': classification_report(y_sent_test, sent_nb_test_pred, output_dict=True, zero_division=0)
}

with open('reports/sentiment_naive_bayes_tfidf_ngram_report.json', 'w') as f:
    json.dump(sent_report, f, indent=2)

print("✓ Sentiment model saved to models/sentiment_naive_bayes_tfidf_ngram.pkl")
print("✓ Sentiment vectorizer saved to models/sentiment_tfidf_vectorizer_ngram.pkl")
print("✓ Sentiment report saved to reports/sentiment_naive_bayes_tfidf_ngram_report.json")

## 18. Final Summary

**Model Performance Summary with N-grams**

In [ ]:
print("\n" + "="*80)
print("TF-IDF N-GRAM + NAIVE BAYES CLASSIFICATION SUMMARY")
print("="*80)

print("\nCATEGORY CLASSIFICATION:")
print(f"  Dataset: {len(documents)} documents")
print(f"  N-gram Range: (1, 2) - Unigrams + Bigrams")
print(f"  Feature Count: {X_train.shape[1]:,} n-grams")
print(f"  Train/Test Split: {X_train.shape[0]}/{X_test.shape[0]}")
print(f"  Test Accuracy: {nb_test_metrics['accuracy']:.4f} ({nb_test_metrics['accuracy']*100:.2f}%)")
print(f"  Test F1-Score: {nb_test_metrics['f1']:.4f}")

print("\nSENTIMENT CLASSIFICATION:")
print(f"  Dataset: {len(sentiment_documents)} documents")
print(f"  N-gram Range: (1, 2) - Unigrams + Bigrams")
print(f"  Feature Count: {X_sent_train.shape[1]:,} n-grams")
print(f"  Train/Test Split: {X_sent_train.shape[0]}/{X_sent_test.shape[0]}")
print(f"  Test Accuracy: {sent_nb_test_metrics['accuracy']:.4f} ({sent_nb_test_metrics['accuracy']*100:.2f}%)")
print(f"  Test F1-Score: {sent_nb_test_metrics['f1']:.4f}")

print("\nKEY FEATURES:")
print("  ✓ TF-IDF vectorization with n-grams (1,2)")
print("  ✓ Multinomial Naive Bayes classifier")
print("  ✓ Captures word sequences and context")
print("  ✓ More discriminative features than unigrams")
print("  ✓ Document frequency filtering")
print("  ✓ Fast training and prediction")

print("\nN-GRAM ADVANTAGES:")
print("  ✓ Word pair features (e.g., 'இலங்கை அரசு')")
print("  ✓ Better context understanding")
print("  ✓ Improved phrase recognition")
print("  ✓ Enhanced classification accuracy")

print("\nSAVED ARTIFACTS:")
print("  Models:")
print("    - models/category_naive_bayes_tfidf_ngram.pkl")
print("    - models/sentiment_naive_bayes_tfidf_ngram.pkl")
print("  Vectorizers:")
print("    - models/category_tfidf_vectorizer_ngram.pkl")
print("    - models/sentiment_tfidf_vectorizer_ngram.pkl")
print("  Reports:")
print("    - reports/category_naive_bayes_tfidf_ngram_report.json")
print("    - reports/sentiment_naive_bayes_tfidf_ngram_report.json")

print("\n" + "="*80)
print("TF-IDF N-GRAM CLASSIFICATION PIPELINE COMPLETED SUCCESSFULLY!")
print("="*80)